<a href="https://colab.research.google.com/github/matinapap/Career-Advisor-Chatbot/blob/main/Career_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Career Advisor Pipeline — Google Colab + Gemini + LangGraph
Detailed description for the accompanying notebook / script

---

## ✨ Overview
This project implements a **complete career guidance system** running on **Google Colab**, leveraging:

- **Gemini (google.generativeai)**
- **LangGraph** and modular agents
- **RAG using FAISS + HuggingFace embeddings**
- **SerpAPI** for course search
- **Gradio GUI**

The system analyzes user profiles, recommends suitable roles, generates learning plans, provides resume feedback (powered by RAG), and conducts mock interviews.

---

## 🧠 Pipeline Agents / Nodes

### 🔍 Profile Analyzer
- Processes the text-based user profile (optionally including a PDF resume).  
- Uses the **Gemini** model to extract skills, experiences, and interests.  
- Returns a **detailed narrative description**.

---

### 💼 Career Suggestion Agent
- Produces at least **two recommended career roles**.  
- Includes justification, required skills, and steps for progression.  
- Supports **fallback parsing** in case of LLM JSON failures.

---

### 🎓 Learning Path Generator
Provides a complete learning plan for a chosen role, including:
- technologies  
- modules  
- duration  
- recommended courses (Udemy / YouTube / Coursera)  
- progression roadmap  

---

### 📄 Resume Advisor (RAG-Powered)
- Uses a **FAISS vectorstore**  
- Employs **all-MiniLM-L6-v2** embeddings  
- Performs similarity search on a local **resume_tips.txt** file  
- Produces targeted resume improvements by combining RAG context + LLM reasoning  

---

### 🗣️ Mock Interview Agent
- Generates **realistic mock interview questions**  
- Provides model answers and improvement tips  
- Adapts to the user profile & chosen role (with RAG support)

---

### 🧭 Personalized Learning Agent
An advanced agent that:
- Extracts **skill gaps**  
- Builds a **personalized learning plan** using `learning_style` & `career_goals`  
- Generates skill keywords and queries **SerpAPI** for online courses  
- Saves user preferences to a JSON file  

---

## 🧩 LangGraph Architecture

The system consists of 3 separate LangGraph pipelines:

### 🔹 Default Flow  
Profile → Role Suggestions → Learning Path

### 🔹 Personalized Flow  
Profile → Role Suggestions → Personalized Learning (with skill gaps & preferences)

### 🔹 Interview Flow  
Profile → Role Suggestions → Resume Feedback (RAG) → Mock Interview

### Google Drive Conection

In [ ]:
#  # Συνδέει το Google Drive με το περιβάλλον του Google Colab για να αποκτήσουμε πρόσβαση σε αρχεία από τον λογαριασμό μας
#  from google.colab import drive
#  drive.mount('/content/drive')

### Dependecies Installation

In [ ]:
# !pip uninstall -y langchain langchain-core langchain-community langchain-openai langchain-text-splitters langgraph pydantic

# !pip install "pydantic>=2.0,<3.0"

# !pip install langchain langchain-community langchain-openai langchain-text-splitters langgraph

# !pip install gradio PyPDF2 faiss-cpu sentence-transformers google-generativeai

### Imports

In [ ]:
import json
import re
import os
import traceback
from typing import TypedDict, Optional, List, Dict
from datetime import datetime
import textwrap

# === Third-party libs commonly used ===
import gradio as gr
import PyPDF2
import requests
from pydantic import BaseModel, Field

# === LangChain ===
from langchain.tools import tool
from langchain_openai import ChatOpenAI # ChatOpenAI moved to langchain_openai in 0.2.x
from langchain_core.messages import HumanMessage # HumanMessage moved to langchain_core.messages
from langgraph.graph import StateGraph, START, END

# Embeddings & FAISS RAG
from langchain_text_splitters import RecursiveCharacterTextSplitter # Moved to langchain_text_splitters
from langchain_core.documents import Document # Moved to langchain_core.documents
from langchain_community.vectorstores import FAISS # Moved to langchain_community.vectorstores
from langchain_community.embeddings import HuggingFaceEmbeddings # Moved to langchain_community.embeddings

from google.colab import userdata
import google.generativeai as genai

# === Global constants ===
PREFS_FILE = "user_personalization.json"
HISTORY_FILE = "/content/drive/MyDrive/career_advisor_files/history.json"
RESUME_TIPS_PATH = "/content/drive/MyDrive/career_advisor_files/resume_tips.txt"

### Api Keys Configuration

In [ ]:
genai.configure(api_key=userdata.get("Gemini_Key"))

# print("Διαθέσιμα μοντέλα:")
# for m in genai.list_models():
#     methods = getattr(m, "supported_generation_methods", [])
#     if "generateContent" in methods:
#         print("✅", m.name)

model = genai.GenerativeModel("gemini-flash-latest")

### Αρχικοποίηση Αρχείου Προτιμήσεων

In [ ]:
# === Initialize preferences file if missing ===
def init_preferences_file():
    if not os.path.exists(PREFS_FILE):
        prefs = {
            "learning_style": "visual",
            "career_goals": "Να εργάζομαι εξ αποστάσεως και να έχω ευέλικτο ωράριο."
        }
        with open(PREFS_FILE, "w", encoding="utf-8") as f:
            json.dump(prefs, f, ensure_ascii=False, indent=2)

def save_user_preferences(learning_style: str, career_goals: str):
    with open(PREFS_FILE, "w", encoding="utf-8") as f:
        json.dump({"learning_style": learning_style, "career_goals": career_goals}, f, ensure_ascii=False, indent=2)

def load_personalization_preferences():
    if not os.path.exists(PREFS_FILE):
        init_preferences_file()
    with open(PREFS_FILE, "r", encoding="utf-8") as f:
        prefs = json.load(f)
    return prefs.get("learning_style", "visual"), prefs.get("career_goals", "")

### Βάση Γνώσης για Βιογραφικά

In [ ]:
# === Load resume tips for RAG ===
if os.path.exists(RESUME_TIPS_PATH):
    try:
        with open(RESUME_TIPS_PATH, "r", encoding="utf-8") as f:
            resume_tips = f.read()
    except Exception:
        resume_tips = ""
else:
    resume_tips = ""

# Build small FAISS store from resume tips
if resume_tips:
    docs = [Document(page_content=resume_tips)]
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    try:
        db = FAISS.from_documents(chunks, embedding)
    except Exception:
        db = None
else:
    db = None

### Helpers

In [ ]:
def gemini_prompt(prompt: str, safety_fallback="(Δεν υπάρχει απάντηση)"):
    try:
        return model.generate_content(prompt).text
    except Exception as e:
        return f"{safety_fallback}\n\n(Σφάλμα: {e})"

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Εξάγει text από PDF (PyPDF2 fallback)."""
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() or ""
    except Exception as e:
        text = f"(Σφάλμα στην ανάγνωση PDF: {e})"
    return text

In [ ]:
def clean_output(raw_text: str) -> str:
    if not raw_text:
        return "(No output)"
    raw_text = str(raw_text)
    if raw_text.strip().startswith("<!DOCTYPE") or raw_text.strip().startswith("<html"):
        import re
        return re.sub(r"<[^>]*>", "", raw_text)
    return raw_text

In [ ]:
# ============ LLM Wrapper ============

def llm_run(prompt: str) -> str:
    """Unified LLM call for Gemini / LangChain."""
    try:
        if hasattr(model, "generate_content"):
            return model.generate_content(prompt).text
        if hasattr(model, "predict"):
            return model.predict(prompt)
        if hasattr(model, "generate"):
            messages = [HumanMessage(content=prompt)]
            return model.generate(messages).generations[0][0].text
        return "(model δεν υποστηρίζεται)"
    except Exception as e:
        return f"(model error: {e})"

### Tools

In [ ]:
# ============ Tools (LangChain-style) ============

@tool
def search_courses(skill: str, max_results: int = 3) -> str:
    """
    Αναζητά online courses για ένα skill (Udemy, Coursera, YouTube) μέσω SerpAPI.
    Επιστρέφει markdown list.
    """
    try:
        api_key = None
        if userdata:
            api_key = userdata.get("SERPAPI_KEY")
        if not api_key:
            api_key = os.environ.get("SERPAPI_KEY")

        if not api_key:
            return "ERROR: Missing SerpAPI key."
        platforms = ["udemy.com", "coursera.org", "youtube.com"]
        results_out = []

        for site in platforms:
            query = requests.utils.requote_uri(f"{skill} course site:{site}")
            url = f"https://serpapi.com/search.json?q={query}&api_key={api_key}&num={max_results}"
            try:
                r = requests.get(url, timeout=8)
                if r.status_code != 200:
                    continue
                data = r.json().get("organic_results", [])
                for item in data[:max_results]:
                    title = item.get("title")
                    link = item.get("link")
                    if title and link:
                        results_out.append(f"- [{title}]({link})")
            except Exception:
                continue

        return "\n".join(results_out) if results_out else f"(Δεν βρέθηκαν courses για το skill: {skill})"
    except Exception as e:
        return f"(search_courses error: {e})"


### Agent Initialization

### Agents

#### Profile Analyzer

In [ ]:
def analyze_profile(user_profile: str) -> str:
    prompt = f"""
Ανάλυσε το παρακάτω προφίλ χρήστη και γράψε ένα αναλυτικό κείμενο στα ελληνικά
που περιγράφει τις δεξιότητες, γνώσεις και ενδιαφέροντα του χρήστη.
Περιέγραψε αναλυτικά και χωρίς λίστες.

Προφίλ:
{user_profile}
"""
    return gemini_prompt(prompt)

#### Career Suggestions

In [ ]:
def suggest_careers(skills_summary: str) -> str:
    prompt = f"""
Με βάση τις παρακάτω δεξιότητες:

{skills_summary}

Γράψε αναλυτικά, σε συνεκτικό κείμενο στα ελληνικά:
- 2 κατάλληλους επαγγελματικούς ρόλους
- Γιατί ταιριάζουν
- Απαραίτητες δεξιότητες
- Τι θα πρέπει να κάνει ο χρήστης για να προχωρήσει
"""
    return gemini_prompt(prompt)

#### Suggest Learning Path

In [ ]:
def suggest_learning_path(role: str) -> str:
    prompt = f"""
Πρότεινε ένα αναλυτικό πλάνο εκμάθησης για τον ρόλο: {role}.
Συμπερίλαβε:
- Τεχνολογίες / έννοιες που πρέπει να μάθει
- Προτεινόμενα online μαθήματα (Coursera, YouTube)
- Εκτιμώμενος χρόνος για κάθε θέμα
Γράψε σε ελληνικό συνεκτικό κείμενο, χωρίς bullets.
"""
    return gemini_prompt(prompt)

#### Suggest Resume Improvements

In [ ]:
def suggest_resume_improvements(resume_text: str) -> str:
    """
    Αξιολογεί το βιογραφικό, κάνει RAG με db αν υπάρχει και επιστρέφει
    αναλυτικά σχόλια και προτάσεις βελτίωσης στα ελληνικά, σε πλήρες κείμενο.
    """
    try:
        context = ""
        if db:
            try:
                relevant = db.similarity_search(resume_text, k=3)
                context = "\n\n".join([d.page_content for d in relevant])
            except Exception:
                context = ""

        prompt = f"""
Αξιολόγησε το παρακάτω βιογραφικό και δώσε **αναλυτικά σχόλια** στα ελληνικά.
Συμπεριέλαβε:
- Τι λειτουργεί καλά και τι όχι
- Προτάσεις βελτίωσης για κάθε ενότητα (Εκπαίδευση, Εργασιακή εμπειρία, Δεξιότητες)
- Συγκεκριμένο κείμενο που μπορεί να προστεθεί ή αντικατασταθεί

Βιογραφικό:
{resume_text}

Οδηγίες / Συμπληρωματικό context:
{context}
"""
        raw = llm_run(prompt)

        return clean_output(raw)

    except Exception as e:
        return f"(Σφάλμα suggest_resume_improvements: {e})"


##### Interview Agent

In [ ]:
def interview_agent(user_profile: str, chosen_role: str, chat_context: str = "", resume_text: Optional[str] = None) -> str:
    """
    Ετοιμάζει mock συνέντευξη με πλήρεις ερωτήσεις και αναλυτικές ενδεικτικές απαντήσεις.
    Η έξοδος είναι human-readable κείμενο, όχι απλώς λίστες.
    """
    try:
        skills_summary = analyze_profile(user_profile + "\n\n" + chat_context)
        rag_context = ""
        if resume_text and db:
            try:
                relevant = db.similarity_search(resume_text, k=3)
                rag_context = "\n\n".join([d.page_content for d in relevant])
            except Exception:
                rag_context = ""

        prompt = f"""
Δημιούργησε μια mock συνέντευξη για τον χρήστη με βάση τα παρακάτω στοιχεία.
Συμπεριέλαβε **αναλυτικές απαντήσεις**, επεξηγήσεις και tips για κάθε ερώτηση.

Προφίλ:
{user_profile}

Ρόλος:
{chosen_role}

Δεξιότητες:
{skills_summary}

Βιογραφικό / context:
{rag_context}

Μορφή:
1. Ερώτηση: ...
   Απάντηση: ...
   (Πρόσθετες συμβουλές ή εξηγήσεις)
"""
        raw = llm_run(prompt)
        return clean_output(raw)

    except Exception as e:
        return f"(Σφάλμα interview_agent: {e})"

#### Personalized Learning

In [ ]:
def personalized_learning(user_skills_text, target_role, personalization_info):
    try:
        prompt_role_desc = f"""
Δώσε περιγραφή για τον ρόλο: {target_role}
Περιέγραψε:
- Τι καθήκοντα έχει
- Ποιες τεχνολογίες / δεξιότητες απαιτούνται
"""
        role_description = gemini_prompt(prompt_role_desc)

        prompt_gap = f"""
Σύγκρινε τις παρακάτω δεξιότητες του χρήστη με τις απαιτήσεις του ρόλου:

--- Δεξιότητες Χρήστη ---
{user_skills_text}

--- Περιγραφή Ρόλου ---
{role_description}

Βρες:
- Ποιες δεξιότητες λείπουν;
- Σε ποιες είναι αρχάριος;
- Ποια είναι τα σημαντικότερα gaps για να καλύψει;
Δώσε ως λίστα.
"""
        skill_gaps = gemini_prompt(prompt_gap)

        prompt_learning = f"""
Ο χρήστης θέλει να γίνει: {target_role}
Δίνει έμφαση σε:
- Learning style: {personalization_info.get('learning_style','-')}
- Career goals: {personalization_info.get('career_goals','-')}

Βοήθησέ τον να μάθει ό,τι του λείπει: {skill_gaps}

Φτιάξε ένα πλάνο εκμάθησης (θεματικό), και δώσε:
- Τεχνολογίες ή έννοιες προς εκμάθηση
- Προτάσεις μαθημάτων από Coursera, YouTube
- Προτεινόμενο χρόνος ανά θέμα
"""
        learning_plan = gemini_prompt(prompt_learning)

        prompt_skills = f"""
Είσαι βοηθός για αναζήτηση μαθημάτων.
Από την ανάλυση δεξιοτήτων εξάγεις έως 4 σημαντικές δεξιότητες.
Μετέτρεψε τις σε σύντομα αγγλικά keywords.
Απάντησε σε JSON, πχ:
{{
  "skills": ["python", "sql"]
}}

Skill gaps:
{skill_gaps}

Ρόλος:
{target_role}
"""
        raw_skills = gemini_prompt(prompt_skills)
        import json, re
        cleaned = re.sub(r'```json|```', '', raw_skills).strip()
        try:
            agent_output = json.loads(cleaned)
            skills = agent_output.get("skills", [])
        except Exception:
            skills = []
        if not skills:
            skills = ["python", "sql", "data analysis", "excel"]

        courses_output = ""
        for skill in skills:
            try:
                results = search_courses.invoke({"skill": skill, "max_results": 3})
            except Exception:
                results = "(Σφάλμα στην αναζήτηση μαθημάτων)"
            courses_output += f"### {skill}\n{results}\n\n"


        return f"""
## 🎯 Ρόλος: {target_role}

## 🔎 Ανάλυση Skill Gaps
{skill_gaps}

---

## 🎓 Προσωποποιημένο Πλάνο Εκμάθησης
{learning_plan}

---

## 🎓 Προτεινόμενα Μαθήματα
{courses_output}
"""
    except Exception as e:
        import traceback
        return f"(Error in personalized_learning_with_courses: {e})\n{traceback.format_exc()}"

### LangGraph Agent Flow

In [ ]:
# === LangGraph Agent Flow ===
class AgentState(TypedDict):
    profile: str
    resume_text: str
    role: str
    skills: str
    suggested_roles: str
    learning_plan: str
    resume_feedback: str
    interview_questions: str
    learning_style: str
    career_goals: str

def node_wrapper(output_key, func, *input_keys):
    """
    Γενικός decorator για κόμβους LangGraph.
    Καλεί τη function με τα input keys από το state και ενημερώνει το output_key.
    """
    def wrapper(state: AgentState):
        try:
            inputs = [state.get(k, "") for k in input_keys]
            state[output_key] = func(*inputs)
        except Exception as e:
            state[output_key] = f"(Σφάλμα στο {output_key}: {e})"
        return state
    return wrapper


# === Agents ===

analyze_profile_node = node_wrapper("skills", analyze_profile, "profile")

def suggest_roles_node(state: AgentState):
    try:
        output = suggest_careers(state.get("skills", ""))
        output_clean = clean_output(output)

        try:
            roles = json.loads(output_clean)
            state["suggested_roles"] = roles
            if not state.get("role") and roles:
                state["role"] = roles[0].get("title", "No role")
        except json.JSONDecodeError:
            state["suggested_roles"] = output_clean
            if not state.get("role"):
                first_role = re.search(r"1\.\s+(.*?)\n", output_clean)
                state["role"] = first_role.group(1).strip() if first_role else "No role"

    except Exception as e:
        state["suggested_roles"] = f"(Σφάλμα στο suggest_roles_node: {e})"
        if not state.get("role"):
            state["role"] = "No role"
    return state


learning_path_node = node_wrapper("learning_plan", suggest_learning_path, "role")

personalized_learning_node = node_wrapper(
    "learning_plan",
    lambda skills, role, learning_style, career_goals: personalized_learning(
        skills,
        role,
        {
            "learning_style": learning_style,
            "career_goals": career_goals
        }
    ),
    "skills", "role", "learning_style", "career_goals"
)


def resume_feedback_node(state: AgentState):
    try:
        if state.get("resume_text"):
            feedback = suggest_resume_improvements(state["resume_text"])
            state["resume_feedback"] = clean_output(feedback)
        else:
            state["resume_feedback"] = "(Δεν υποβλήθηκε βιογραφικό)"
    except Exception as e:
        state["resume_feedback"] = f"(Σφάλμα resume_feedback_node: {e})"
    return state

interview_node = node_wrapper(
    "interview_questions",
    lambda profile, role, resume_text: clean_output(
        interview_agent(profile, role, chat_context="", resume_text=resume_text or profile)
    ),
    "profile", "role", "resume_text"
)


# === Graphs ===

# Default Flow
builder_default = StateGraph(AgentState)
builder_default.add_node("AnalyzeProfile", analyze_profile_node)
builder_default.add_node("SuggestRoles", suggest_roles_node)
builder_default.add_node("LearningPath", learning_path_node)
builder_default.add_edge("AnalyzeProfile", "SuggestRoles")
builder_default.add_edge("SuggestRoles", "LearningPath")
builder_default.add_edge("LearningPath", END)
builder_default.set_entry_point("AnalyzeProfile")
default_graph = builder_default.compile()

# Personalized Flow
builder_personalized = StateGraph(AgentState)
builder_personalized.add_node("AnalyzeProfile", analyze_profile_node)
builder_personalized.add_node("SuggestRoles", suggest_roles_node)
builder_personalized.add_node("PersonalizedLearning", personalized_learning_node)
builder_personalized.add_edge("AnalyzeProfile", "SuggestRoles")
builder_personalized.add_edge("SuggestRoles", "PersonalizedLearning")
builder_personalized.add_edge("PersonalizedLearning", END)
builder_personalized.set_entry_point("AnalyzeProfile")
personalized_graph = builder_personalized.compile()

# Interview Flow
builder_interview = StateGraph(AgentState)
builder_interview.add_node("AnalyzeProfile", analyze_profile_node)
builder_interview.add_node("SuggestRoles", suggest_roles_node)
builder_interview.add_node("ResumeFeedback", resume_feedback_node)
builder_interview.add_node("Interview", interview_node)
builder_interview.add_edge("AnalyzeProfile", "SuggestRoles")
builder_interview.add_edge("SuggestRoles", "ResumeFeedback")
builder_interview.add_edge("ResumeFeedback", "Interview")
builder_interview.add_edge("Interview", END)
builder_interview.set_entry_point("AnalyzeProfile")
interview_graph = builder_interview.compile()


### Pipeline

In [ ]:
# ============ Orchestrator: full_pipeline (like original but agentic) ============

def full_pipeline(
    user_profile: str,
    chosen_role: str = "",
    resume_file=None,
    chat_history_messages: Optional[list] = None,
    mode = "default",
    learning_style: Optional[str] = None,
    career_goals: Optional[str] = None,
    personalized: bool = False
):
    # Load personalization defaults if needed
    ls, cg = load_personalization_preferences()
    learning_style = learning_style or ls
    career_goals = career_goals or cg

    # === Επεξεργασία Βιογραφικού (PDF) ===
    resume_text = ""
    resume_clean = ""
    if resume_file:
        try:
            resume_text = extract_text_from_pdf(resume_file.name)
            resume_clean = clean_output(resume_text)
            user_profile += f"\n\n{resume_clean}"
        except Exception as e:
            resume_text = f"(Σφάλμα PDF: {e})"
            resume_clean = resume_text

    # initial state
    initial_state: AgentState = AgentState(
        profile=user_profile,
        resume_text=resume_text,
        role=chosen_role or "",
        skills="",
        suggested_roles="",
        learning_plan="",
        resume_feedback="",
        interview_questions="",
        learning_style=learning_style,
        career_goals=career_goals
    )

   # ============== Επιλογή Graph ανά mode ==============
    if mode == "default":
      final_state = default_graph.invoke(initial_state)

      if chosen_role.strip():
          result_markdown = f"""
  ## 🧠 Ανάλυση Δεξιοτήτων
  {final_state['skills']}

  ## 🎓 Προτεινόμενο Πλάνο Εκμάθησης
  {final_state['learning_plan']}
  """
      else:
          result_markdown = f"""
  ## 🧠 Ανάλυση Δεξιοτήτων
  {final_state['skills']}

  ## 💼 Προτεινόμενοι Ρόλοι
  {final_state['suggested_roles']}

"""

    elif mode == "personalized":
      final_state = personalized_graph.invoke(initial_state)
      save_user_preferences(learning_style, career_goals)
      result_markdown = final_state["learning_plan"]
    elif mode == "interview":
      final_state = interview_graph.invoke(initial_state)
      parts = []

      # Resume feedback
      resume_feedback = final_state.get("resume_feedback") or "(Δεν υποβλήθηκε ή σφάλμα στο βιογραφικό)"
      parts.append(f"## ✍️ Σχόλια για το Βιογραφικό\n{resume_feedback}")

      # Interview questions
      interview_text = final_state.get("interview_questions") or "(Δεν υπάρχουν ερωτήσεις συνέντευξης)"
      parts.append(f"## 🎤 Mock Συνέντευξη\n{interview_text}")

      result_markdown = "\n\n".join(parts)
    else:
      result_markdown = "⚠️ Μη υποστηριζόμενη λειτουργία."
      final_state = {}

    print(result_markdown)

    return result_markdown, final_state


### GUI

In [ ]:
# ============ Gradio UI (similar layout to your original) ============

def toggle_personalization_fields(mode):
    show = mode == "personalized"
    if show:
        learning_style, career_goals = load_personalization_preferences()
        return (
            gr.update(visible=True, value=learning_style),
            gr.update(visible=True, value=career_goals)
        )
    return gr.update(visible=False), gr.update(visible=False)

init_preferences_file()
default_learning_style, default_career_goals = load_personalization_preferences()

with gr.Blocks(css=".gr-button { width: 100% !important; }") as demo:
    output_md = gr.Markdown(label="📊 Αποτελέσματα")

    with gr.Row():
        profile = gr.Textbox(lines=8, label="🔍 Προφίλ / Περιγραφή", interactive=True)

    with gr.Row():
        role = gr.Textbox(lines=1, label="🎯 Επιλεγμένος Ρόλος (προαιρετικά)", interactive=True)

    with gr.Row():
        resume = gr.File(label="📄 Βιογραφικό (PDF)", file_types=[".pdf"])

    with gr.Row():
        mode_dropdown = gr.Dropdown(
            ["default", "interview", "personalized"],
            label="Λειτουργία",
            value="default"
        )

    with gr.Row():
        learning_style_input = gr.Dropdown(
            choices=["visual", "auditory", "hands-on"],
            label="📚 Στυλ Μάθησης",
            value=default_learning_style,
            visible=False
        )

        career_goals_input = gr.Textbox(
            label="🎯 Στόχοι Καριέρας",
            value=default_career_goals,
            visible=False
        )

    mode_dropdown.change(
        toggle_personalization_fields,
        inputs=[mode_dropdown],
        outputs=[learning_style_input, career_goals_input]
    )

    state = gr.State()

    with gr.Row():
        submit_btn = gr.Button("🚀 Submit", scale=1)

    with gr.Row():
        clear_btn = gr.Button("🧹 Clear", scale=1)

    submit_btn.click(
    fn=full_pipeline,
          inputs=[
              profile, role, resume, state,
              mode_dropdown,
              learning_style_input,
              career_goals_input
          ],
          outputs=[output_md, state]
      )

    clear_btn.click(
        fn=lambda: ("", []),
        outputs=[output_md, state]
    )

### Execution

In [ ]:
if __name__ == "__main__":
    demo.launch(share=True, debug=True)